In [1]:
import pyscf
import pyscf.tools
from pyscf import lo, gto


from orbitalpartitioning import *


molecule = """
 Fe 1.67785607 0.00052233 0.06475932
 O 0.00000000 0.00000000 -0.47099074
 Fe -1.67785607 -0.00052233 0.06475932
 Cl 1.87002704 -1.09796437 1.99091682
 Cl 2.93244917 -0.98210488 -1.47467288
 Cl 2.37160936 2.07954091 -0.50446591
 Cl -1.87002704 1.09796437 1.99091682
 Cl -2.93244917 0.98210488 -1.47467288
 Cl -2.37160936 -2.07954091 -0.50446591
 """

pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   False,
        spin    =   10, # number of unpaired electrons
        charge  =   -2)
pymol.build()

basis = "def2-svp"
mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "minao"
mf.run()
##################################################################




******** <class 'pyscf.scf.rohf.ROHF'> ********
method = ROHF-RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 0 MB)
num. doubly occ = 77  num. singly occ = 10
init E= -5289.63018546547
  HOMO = 0.473246078347928  LUMO = 0.476436886667598
cycle= 1 E= -5289.15069324983  delta_E= 0.479  |g|= 2.84  |ddm|= 8.23
  HOMO = -0.000303795913073977  LUMO = 0.0865952448133458
cycle= 2 E= -5276.15095510045  delta_E=   13  |g|= 4.14  |ddm|= 9.58

WARN: HOMO 0.91237260160555 >= LUMO 0.129031757301459

cycle= 3 E= -5291.20851605662  delta_E= -15.1  |g|= 2.39  |ddm|= 9.69
  HOMO = 0.192121528469619  LUMO = 0.454224553216574
cycle= 4 E= -5297.24364203003  delta_E= -6.04  |g|= 1.78  |ddm|= 8.96
  HOMO = 0.193863066010502  LUMO = 0.42795

ROHF-RHF object of <class 'pyscf.scf.rohf.ROHF'>

# ROHF/def2-tzvp

In [ ]:

pymol.basis = "def2-tzvp"
pymol.build()
mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "chkfile"
mf.run()
##################################################################


In [2]:

#
#   Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()
Cdocc = mf.mo_coeff[:,mf.mo_occ==2]
Csing = mf.mo_coeff[:,mf.mo_occ==1]
Cvirt = mf.mo_coeff[:,mf.mo_occ==0]
ndocc = Cdocc.shape[1]
nsing = Csing.shape[1]
nvirt = Cvirt.shape[1]

print(" Norbs: ndocc = %6i nsing = %6i nvirt = %6i"%(ndocc, nsing, nvirt))

 Norbs: ndocc =     77 nsing =     10 nvirt =      8


In [3]:
# Find AO's corresponding to atoms
full = []
frag1 = []
frag2 = []
frag3 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 0:
        if ao[2] in ("3d",):
            frag1.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 1:
        if ao[2] in ("2p",):
            frag2.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 2:
        if ao[2] in ("3d",):
            frag3.append(ao_idx)
            full.append(ao_idx)


frags = [frag1, frag2, frag3]
print(frags)

[[13, 14, 15, 16, 17], [20, 21, 22], [36, 37, 38, 39, 40]]


In [4]:
#
#   Get full active space
Cenv, Oact, Vact, Venv = dmet_active_space(Cdocc, Cvirt, full, S)


#
#   Split active space into fragments
init_fspace = []
clusters = []
Cfrags = []
orb_index = 1


for fi,f in enumerate(frags):
    print()
    print(" Fragment: ", f)
    (Of, Sf, Vf), (_, _, _) = svd_subspace_partitioning_nonorth((Oact, Sact, Vact), f, S)

    Cfrags.append(np.hstack((Of, Sf, Vf)))
    ndocc_f = Of.shape[1]
    init_fspace.append((ndocc_f+Sf.shape[1], ndocc_f))
    nmof = Of.shape[1] + Sf.shape[1] + Vf.shape[1]
    clusters.append(list(range(orb_index, orb_index+nmof)))
    orb_index += nmof


# Orthogonalize Fragment orbitals
Cfrags = sym_ortho(Cfrags, S)

# Pseudo canonicalize fragments
Cfrags = canonicalize(Cfrags, F)


Cact = np.hstack(Cfrags)

print(" sing vals of active space: ", np.linalg.svd(Cact.T @ S @ Cact)[1])
# Write Molden files for visualization
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
pyscf.tools.molden.from_mo(mf.mol, "Cenv.molden", Cenv)
for i in range(len(frags)):
    di = Cfrags[i] @ Cfrags[i].T
    pyscf.tools.cubegen.density(mf.mol, 'fragden_{:02d}.cube'.format(i+1), di)


print(" init_fspace = ", init_fspace)
print(" clusters    = ", clusters)


 In svd_subspace_partitioning_nonorth
 Partition   95 orbitals into a total of   13 orbitals
            Index   Sing. Val. Space       
                0   0.99848339            1*
                1   0.99809506            1*
                2   0.99749530            1*
                3   0.99699292            1*
                4   0.99212916            1*
                5   0.99205547            1*
                6   0.98550082            1*
                7   0.98463040            1*
                8   0.98122638            1*
                9   0.97228914            1*
               10   0.96422451            0*
               11   0.95575784            0*
               12   0.94515475            0*
               13   0.37853680            2
               14   0.30767982            2
               15   0.27180705            2
               16   0.16217179            0
               17   0.16063835            0
               18   0.14681707            0
              

# Build exchange matrix 

In [ ]:
# Build exchange matrix 
Ka,Kb = mf.get_k()
Ksing_docc = Csing.T@(Ka+Kb)@Cdocc
Ksing_virt = Csing.T@(Ka+Kb)@Cvirt

# Build active space

In [ ]:

U,s,Vh = np.linalg.svd(Ksing_docc, full_matrices=True)
print(Vh.shape)
print(" singular values: <sing|K|docc>")
for si in s:
    print(" %12.8f"%si)
Cdocc2 = Cdocc @ Vh
    
U,s,Vh = np.linalg.svd(Ksing_virt, full_matrices=True)
print(" singular values: <sing|K|virt> ")
for si in s:
    print(" %12.8f"%si)
Cvirt2 = Cvirt @ Vh


Cact = np.hstack((Cdocc2[:,0:nsing], Csing, Cvirt2[:,0:nsing]))
Cdocc_new = Cdocc2[:,nsing:-1]
Cvirt_new = Cvirt2[:,nsing:-1]

print(" ndoc: ", Cdocc_new.shape)
print(" nact: ", Cact.shape)
print(" nvir: ", Cvirt_new.shape)

print(np.linalg.norm(Cdocc_new.T @ S @ Cact))
print(np.linalg.norm(Cvirt_new.T @ S @ Cact))
print(np.trace(Cdocc_new.T @ S @ Cact))
print(np.trace(Cvirt_new.T @ S @ Cact))
pyscf.tools.molden.from_mo(mf.mol, "scr/Cact.molden", Cact)
pyscf.tools.molden.from_mo(mf.mol, "scr/Cdoc.molden", Cdocc_new)

Cact_loc = pyscf.lo.PM(pymol).kernel(Cact)
pyscf.tools.molden.from_mo(mf.mol, "scr/Cact_loc.molden", Cact_loc)

# Make clusters

In [ ]:
init_fspace = []
clusters = []

# Clusters

# extra
clusters.append([0,])
# left Cl
clusters.append([2,])
clusters.append([6,])
clusters.append([7,])
# left Fe
clusters.append([10,12,16,18,19,21,23,25,27,29])
# oxygen 
clusters.append([3,5,8])
# right Fe 
clusters.append([11,13,14,15,17,20,22,24,26,28])
# right Cl 
clusters.append([1,])
clusters.append([4,])
clusters.append([9,])

init_fspace = []
# extra
init_fspace.append((1,1))
# left Cl
init_fspace.append((1,1))
init_fspace.append((1,1))
init_fspace.append((1,1))
# left Fe
init_fspace.append((5,0))
# oxygen 
init_fspace.append((3,3))
# right Fe 
init_fspace.append((0,5))
# right Cl 
init_fspace.append((1,1))
init_fspace.append((1,1))
init_fspace.append((1,1))

order = []
for ci in clusters:
    for cii in ci:
        order.append(cii)

Cact_ordered = Cact_loc[:,order]
pyscf.tools.molden.from_mo(mf.mol, "scr/Cact_ordered.molden", Cact_ordered)

# Make integrals

In [ ]:

Cenv = Cdocc_new * 1.0

#   Make integrals
#
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact_ordered.T @ h @ Cact_ordered;
j = Cact_ordered.T @ j @ Cact_ordered;
k = Cact_ordered.T @ k @ Cact_ordered;
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact_ordered, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact_ordered @ Cact_ordered.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;

np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Cact_ordered)
np.save("overlap_mat", S)

Pa = mf.make_rdm1()[0]
Pb = mf.make_rdm1()[1]
np.save("Pa", Cact_ordered.T @ S @ Pa @ S @ Cact_ordered)
np.save("Pb", Cact_ordered.T @ S @ Pb @ S @ Cact_ordered)


In [ ]:
j = 1
clusters_out = []
for ci in clusters:
    c = []
    for i in range(j,j+len(ci)):
        c.append(i)
        j+=1
    clusters_out.append(c)

print("clusters = ", clusters_out)
print("init_fspace = ", init_fspace)
